In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

from hydra import compose, initialize
import numpy as np
import os
import sys
from tqdm.auto import tqdm

sys.path.append('../..')
from data_utils import get_dsa_run_list

In [5]:
with initialize(version_base="1.3", config_path="../../DSA_comparisons/conf"):
    cfg = compose(config_name="config")

In [6]:
data_class = cfg.params.data_class
session_list = [f[:-4] for f in os.listdir(os.path.join(cfg.params.all_data_dir, data_class)) if f.endswith('.mat')]

In [7]:
session_list = [session_list[1], session_list[2]]

In [8]:
session_list

['MrJones-Anesthesia-20160210-01', 'Mary-Anesthesia-20160818-02']

In [9]:
cfg.params.stride

60

# Run DSA

In [10]:
areas = ['all']

In [11]:
all_indices_to_run = {}
for session in session_list:
    print("-"*20)
    print(f"SESSION = {session}")
    print("-"*20)
    dsa_run_list = get_dsa_run_list(session, os.path.join(cfg.params.dsa_results_dir, cfg.params.data_class), cfg.params.grid_search_results_dir, cfg.params.all_data_dir, T_pred=cfg.params.T_pred, stride=cfg.params.stride)
    normed_folder = 'NOT_NORMED' if not cfg.params.normed else 'NORMED'
    filter_folder = f"[{cfg.params.high_pass},{cfg.params.low_pass}]" if cfg.params.low_pass is not None or cfg.params.high_pass is not None else 'NO_FILTER'
    
    for area in areas:
        save_dir = os.path.join(cfg.params.dsa_results_dir, cfg.params.data_class, 'dsa_results', session, normed_folder, f"SUBSAMPLE_{cfg.params.subsample}", filter_folder, f"STRIDE_{cfg.params.stride}", area)

        os.makedirs(save_dir, exist_ok=True)
        
        saved_files = os.listdir(save_dir)
        indices_to_run = []
        for run_index in range(len(dsa_run_list[area])):
            filename = f"run_index-{run_index}.pkl"
            if filename not in saved_files:
                indices_to_run.append(run_index)
        
        if len(indices_to_run) == 0:
            print(f"*COMPLETE*: All results completed for {session} - {area}!!")
        elif len(indices_to_run) == len(dsa_run_list[area]):
            print(f"NOT STARTED: no results completed for {session} - {area}. Adding all indices! ({len(indices_to_run)})")
            if session not in all_indices_to_run:
                all_indices_to_run[session] = {}
            all_indices_to_run[session][area] = indices_to_run
        else:
            print(f"INCOMPLETE: {session} - {area} incomplete, adding indices {indices_to_run}")
            if session not in all_indices_to_run:
                all_indices_to_run[session] = {}
            all_indices_to_run[session][area] = indices_to_run

--------------------
SESSION = MrJones-Anesthesia-20160210-01
--------------------
INCOMPLETE: MrJones-Anesthesia-20160210-01 - all incomplete, adding indices [400, 401, 402, 403, 404, 405, 406, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 

# Running

In [12]:
os.chdir('/om2/user/eisenaj/code/UniversalUnconsciousness')

In [13]:
batch_size = 200

In [ ]:
sessions_to_run = list(all_indices_to_run.keys())
# sessions_to_run = sessions_to_run[:4]

# iterator = tqdm(total=np.sum([len(all_indices_to_run[session]) for session in sessions_to_run]), desc='Hydra Multiprocessing - DSA on Neural Data')
iterator = tqdm(total=np.sum([np.sum([int(np.ceil(len(all_indices_to_run[session][area])/batch_size)) for area in all_indices_to_run[session].keys()]) for session in sessions_to_run]), desc='Hydra Multiprocessing - DSA on Neural Data')

for session in sessions_to_run:
    for area in all_indices_to_run[session].keys():
        print(f"Running indices for {session} - {area}")
        num_batches = int(np.ceil(len(all_indices_to_run[session][area])/batch_size))
        for batch_num in range(num_batches):
            batch_start = batch_num*batch_size
            batch_end = np.min([(batch_num + 1)*batch_size, len(all_indices_to_run[session][area])])
            print(f"running batch #{batch_num}")
            os.system(f"HYDRA_FULL_ERROR=1 python DSA_comparisons/run_dsa.py -m ++params.session={session} ++params.area={area} ++params.run_index={','.join([str(i) for i in all_indices_to_run[session][area][batch_start:batch_end]])}")
            iterator.update()
iterator.close()

Hydra Multiprocessing - DSA on Neural Data:   0%|          | 0/19 [00:00<?, ?it/s]

Running indices for MrJones-Anesthesia-20160210-01 - all
running batch #0
[2024-06-25 14:52:38,068][HYDRA] Submitit 'slurm' sweep output dir : multirun/2024-06-25/14-52-20
[2024-06-25 14:52:38,070][HYDRA] 	#0 : ++params.session=MrJones-Anesthesia-20160210-01 ++params.area=all ++params.run_index=400
[2024-06-25 14:52:38,075][HYDRA] 	#1 : ++params.session=MrJones-Anesthesia-20160210-01 ++params.area=all ++params.run_index=401
[2024-06-25 14:52:38,079][HYDRA] 	#2 : ++params.session=MrJones-Anesthesia-20160210-01 ++params.area=all ++params.run_index=402
[2024-06-25 14:52:38,083][HYDRA] 	#3 : ++params.session=MrJones-Anesthesia-20160210-01 ++params.area=all ++params.run_index=403
[2024-06-25 14:52:38,087][HYDRA] 	#4 : ++params.session=MrJones-Anesthesia-20160210-01 ++params.area=all ++params.run_index=404
[2024-06-25 14:52:38,092][HYDRA] 	#5 : ++params.session=MrJones-Anesthesia-20160210-01 ++params.area=all ++params.run_index=405
[2024-06-25 14:52:38,096][HYDRA] 	#6 : ++params.session=MrJo